<div style="background: linear-gradient(to right, #56BEB7 0%, #56BEB7 50%, #2B317F 50%, #2B317F 100%);"><br></div>

<img src="oathlink_logo.png" width="150" align="right"/>

# Tutorial Oathlink

## Flujo de información

La ley 19.628 sobre la protección de la vida privada define lineamientos para la implementación del Sistema de Finanzas Abiertas (SFA) intoducido por la ley Fintech.

Crucialmente el cliente final (persona natural o juridica) es el dueño de sus datos.

Las instituciones financieras (tales como los bancos) son meros custodios de dicha información.

En consequencia, el dueño tiene el derecho de solicitarle al custodio la entrega de su informacion personal y potencialmente sensible, por ejemplo los movimientos de su cuenta corriente a un usuario (tipicamente a una Fintech) que le brindaría un servicio basado en est información.

Oathlink interviene en la entrega de estos datos personales de parte del custodio al usuario, garantizando minimos de seguidad y velando el cumplimiento con la ley 19.628.

## I. Hello world!

### I.A. Archivos entregados por Oathlink

El equipo de Oathlink confirma la inscripción de una nueva institucion (custodio o usuario) con la entrega de:

- *oathSecret.key*: Archvio que contiene una clave de encriptación.
- *oathlink.pem y oathlink.key*: Archivos de autentificación y certificación de Oathlink que la nueva institución deberá utilizar para generar su certificado de autentificación Mutual TLS con OpenSSL.

### I.B. Generación del certificado Mutual TLS

#### 1) Crear la clave privada del certificado:

    openssl genrsa -out oathCertificate.key 2049

Este comando generara un archivo oathCertificate.key que no deberá ser divulgado.

#### 2) Generar la soliciud de firma de certificado (CSR):

La solicitud de firma de certificadi (Certificate Signing Request en inglés) almacena los datos de la nueva institución.

    openssl req -new -key private.key -cert contact.csr

Este comando genera la solicitud de firma contact.csr usando la clave privada anteriormente creada.

Es crucial completar estos datos, en particular el campo "Common Name" con el RUT bajo el formato 12345678-9.

La constraseña debe permanecer vacia.

#### 3) Firmar el certificado:

    openssl x509 -req -in contact.csr -CA oathlink.pem -CAkey oathlink.key -set_serial 01 -out oathCertificate.pem -days 3650 -sha_256

Este comando genera el certificado oathCertificate.pem que será utilizado para la conexión segura. Este certificado sera firmado con los datos contact.csr y las claves CA de Oathlink.

### I.C. Comprobación de la conexión

Para poder comprobar la conexion se debera instalar el cliente de Python de Oathlink. Para esto usaremos el siguiente comando:

In [ ]:
!pip install oathlink

Es importante notar que el carácter "!" antes del comando es exclusivo para ejecutar comandos desde el ambiente Jupyter, en una consola estándar no es necesario incluirlo.

Para poder comprobar la conexion se debe importar el paquete una vez instalado:

In [ ]:
import oathlink

Una vez importada la libreria, se debe especificar las rutas de los archivos de certificado y key:

In [ ]:
certificateFilename = 'oathCertificate.pem'
keyFilename = 'oathCertificate.key'

Luego debemos usar la funcion de prueba de la libreria, llamada "helloOathlink", como se muestra en el siguiente cuadro:

In [ ]:
response = oathlink.helloOathlink(certificateFilename=certificateFilename, keyFilename=keyFilename)
print(response)

Si todo esta en orden, obtendremos el mesaje de saludo de parte de Oathlink, de lo contrario recibiremos un error que nos informara que puede faltar para obtener acceso.

## II. Entrega de la información por el custodio

### II.A. Solicitar el URL de carga

El protocolo de Oathlink garantiza que la información sea cargada unicamente por el custodio y descargada solamente por el usuario designado.

El primer paso es la solicitud del URL de carga, para esto usaremos la funcion del cliente Python llamada "upload", como se muestra a continuación:

In [ ]:
import json

ownerId = "23109650-7"
userId = "76136279-8"
ownerAuthorization = ""
description = "First test."
intent = "Testing."
uploadResponse = oathlink.upload(certificateFilename=certificateFilename, keyFilename=keyFilename, userId=userId,
                           ownerId=ownerId, ownerAuthorization=ownerAuthorization, description=description,
                           intent=intent)
print(uploadResponse)

# Extract information from response

uploadResponse = json.loads(uploadResponse)
url = uploadResponse.get("oath_link")
oathIdEncrypted = uploadResponse.get("oath_id")

Asi el custodio recibe tambien el identificador "oathId" encryptado que se debe enviar al usuario. El usuario luego desencriptará el oathId y procederá a la descarga. La carga y descarga pueden ser asincronicas.

Oathlink registra la iniciación del intercambio de información entre el custodio y el usuario.

### II.B. Carga de la información

La informacion se carga al url mediante el protocolo PUT (HTTP). La carga esta habilitada solamente para los IP previamente declarados:

In [ ]:
file_path = 'test.txt'
response = oathlink.data_upload(url, file_path)
print(response)

### II.C. Cancelar el intercambio

En caso de emergencia, el custodio puede cancelar la entrega de informacion, para esto debera usar el valor "oathSecret" que se entregara al crear sus credenciales:

In [ ]:
response = oathlink.cancel(certificateFilename=certificateFilename, keyFilename=keyFilename, recordId=recordId)
print(response)

Esta cancelación es imposible despues del acceso por el usuario.

## III. Recepción de la información por el usuario

### III.A. Desencriptar el identificador

El custodio no entrega la información solicitada de manera directa, sino un identificador iathId encriptado por Oathlink con la clave oathlinkSecret.key del usuario.

El primer paso es la desencriptación de este identificador mediante el algoritmo Fernet con el fin de entender el oathId de la información solicitada:

In [ ]:
oathSecret = 'S4KQqEzBRGyZqsdm3qMmCTB9AnHsrWazzm574jLH8HA='
oathId = oathlink.decrypt(oathIdEncrypted=oathIdEncrypted, oathSecret=oathSecret)
print(oathId)